In [1]:
import ROOT
import numpy as np


In [2]:
mu = ROOT.TFile.Open("../build/Mu_1.root")

# Get the TTrees
mu_ntuple = mu.Get("PixelEDep") 
print(mu_ntuple)

Name: PixelEDep Title: Energy per pixel


In [17]:
br_mu = [branch.GetName() for branch in mu_ntuple.GetListOfBranches()]
print(br_mu)

['EventID', 'PixelID', 'EnergyDep']


In [3]:
df = ROOT.RDataFrame("PixelEDep", "../build/Mu_1.root")
data = df.AsNumpy(['EventID', 'PixelID', 'EnergyDep'])  

In [4]:
copyno = data['PixelID']
en = data['EnergyDep']
print(len(en))

20124


In [ ]:
hist = ROOT.TH1F("mu", "Muon;En(MeV);Counts", 200, 0, 15)
canvas = ROOT.TCanvas("canvas", "Canvas", 800, 600)
for value in en:
    hist.Fill(value)

# Fit with Landau
#fit_result = hist.Fit("landau", "RS", "", 0.6, 1.5) # "S" returns the fit result
#landau = fit_result.Get()  
# Extract MPV and sigma
#mpv   = landau.Parameter(1)
#sigma = landau.Parameter(2)
#chi2  = landau.Chi2()
#ndf   = landau.Ndf()
#canvas.SetLogy()
#print(f"MPV: {mpv:.3f} MeV")
#print(f"Sigma: {sigma:.3f}")
#print(f"Chi2/NDF: {chi2:.2f} / {ndf}")  
#hist.Draw("E")  # "E" for error bars
#canvas.Update()    

In [6]:
landau = ROOT.TF1("landau", "landau", 0.55, 1.1)
gaus = ROOT.TF1("gaus", "gaus", -1, 1)
conv = ROOT.TF1Convolution("landau", "gaus", 0.55, 1.1, True)
#conv.SetNofPointsFFT(1000)  # improve resolution of FFT
f_conv = ROOT.TF1("f_conv", conv, 0.55, 1.1, conv.GetNpar())
f_conv.SetParameters(0.1, 1.0, 0.2, 0.0, 0.3) 

hist.Fit(f_conv, "R")  # "R" means fit in range

# Draw result
#canvas = ROOT.TCanvas("c", "Landau-Gauss Fit", 800, 600)
hist.Draw()
f_conv.SetLineColor(ROOT.kRed)
f_conv.Draw("same")
canvas.SaveAs("landau_gauss_fit.png")


****************************************
         Invalid FitResult  (status = 4 )
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      8.39278
NDf                       =            6
Edm                       =  8.64658e-05
NCalls                    =         1636
p0                        =      7253.56   +/-   15482       
p1                        =      1.33957   +/-   1.49325     
p2                        =      0.95271   +/-   3.4587      
p3                        =     -3.85327   +/-   4.37517     
p4                        =      1.84991   +/-   1.15704     


Error in <TVirtualFFT::FFT>: handler not found
Error in <TVirtualFFT::FFT>: handler not found
Warning in <TF1Convolution::MakeFFTConv>: Cannot use FFT, probably FFTW package is not available. Switch to numerical convolution
Warning in <Fit>: Abnormal termination of minimization.
Info in <TCanvas::Print>: png file landau_gauss_fit.png has been created


In [5]:
# f_conv is your fitted TF1 object
mpv = f_conv.GetMaximumX()
print(f"MPV (from maximum of fit): {mpv:.3f} MeV")


NameError: name 'f_conv' is not defined

In [7]:
print(len(en[en<0.55])/len(en))

0.1592133533431943
